In [ ]:
!pip install lmfit

     |████████████████████████████████| 266kB 10.0MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 256kB 26.4MB/s 
  Created wheel for lmfit: filename=lmfit-1.0.1-cp36-none-any.whl size=81990 sha256=c2cfc5969024fa5972e5f2fb453526dcd8cd5844a77f40dc9b11292ebf4eb3d7
  Stored in directory: /root/.cache/pip/wheels/ef/22/8d/6cd9c0af36f501cfb248673b05dad5d2c27504951d55519acb
  Created wheel for asteval: filename=asteval-0.9.21-cp36-none-any.whl size=17285 sha256=2eeaf3549a83e9c4e28f1920911916445b8df6a2cc3bcbd436935df6b860036e
  Stored in directory: /root/.cache/pip/wheels/53/9d/3b/da1ff15feb7d97bb195837f4e78f971c748a945bd4720003ce
Successfully built lmfit asteval


In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import plotly.graph_objects as go
import plotly.io as pio
import requests
from lmfit import minimize, Parameters, Parameter, report_fit

In [ ]:
path = ""

In [ ]:
# from google.colab import drive 
# drive.mount('/content/drive',force_remount=True)
# path = "/content/drive/MyDrive/209 Project/"

Mounted at /content/drive


### Please make sure you run the naive model first and has naive_model_per_county.csv and naive_all_preds_test.csv in your data/ folder 

In [ ]:
def ode_model(z, t, beta, sigma, gamma):
    """
    Reference https://github.com/silpara/simulators/blob/master/compartmental_models
    """
    S, E, I, R = z
    N = S + E + I + R
    dSdt = -beta*S*I/N
    dEdt = beta*S*I/N - sigma*E
    dIdt = sigma*E - gamma*I
    dRdt = gamma*I
    return [dSdt, dEdt, dIdt, dRdt]

In [ ]:
def ode_solver(t, initial_conditions, params):
    """
    Reference https://github.com/silpara/simulators/blob/master/compartmental_models
    """
    initE, initI, initR, initN = initial_conditions
    beta, sigma, gamma = params['beta'].value, params['sigma'].value, params['gamma'].value
    initS = initN - (initE + initI + initR)
    res = odeint(ode_model, [initS, initE, initI, initR], t, args=(beta, sigma, gamma))
    return res

In [ ]:
# read the confirmed cases data and population data
covid_confirmed = pd.read_csv(path + 'data/covid_cases/covid_confirmed_usafacts_new.csv')
popu =  pd.read_csv(path +'data/covid_cases/covid_county_population_usafacts.csv')
covid_confirmed = covid_confirmed.dropna(axis=0,how='any') 

# read the naive results, for comparasion and ensemble
naive_model_per_county = pd.read_csv(path +'data/naive_model_per_county.csv')
naive_daily = pd.read_csv(path +'data/naive_all_preds_test.csv')
fips = naive_model_per_county['fip'].tolist()

In [ ]:
# the seir parameters
params = Parameters()
params.add('beta', value=1, min=0, max=10)
params.add('sigma', value=1, min=0, max=10)
params.add('gamma', value=1, min=0, max=10)

In [ ]:
pred_error = []
per_fip = []
per_county = []
per_state = []
fit_error = []
ensemble_error = []

all_preds_test = pd.DataFrame(columns=['fip','County Name','State','date','seir_orig_pred','seir_ensemble_pred'])

# helper function to calculate MAPE. 
def div0( a, b ):
    a = a.astype(np.float)
    b = b.astype(np.float)
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
        c[c == np.inf] = 1
        c = np.nan_to_num(c)  # -inf inf NaN
    return c

In [ ]:
# do prediction for all counties
for fip in fips:
    if fip==0:
        continue
    # Get one county data
    one_county = covid_confirmed[covid_confirmed['countyFIPS']==fip]
    one_county = one_county.T.reset_index().iloc[4:,:]
    one_county.columns = ['date','daily_confirmed']
    name = covid_confirmed[covid_confirmed['countyFIPS']==fip]['County Name'].values[0]
    state = covid_confirmed[covid_confirmed['countyFIPS']==fip]['State'].values[0]
    # Get Sept and Oct data
    one_county = one_county.iloc[-70:-9,:].reset_index(drop=True)
    
    # Init N E I R
    # N uses the population
    initN = popu[popu['countyFIPS']==fip]['population'].values
    # We assume E is the confirmed cases*5
    initE = one_county.iloc[0,1]*5
    # According to the fact that overall recovery and death proportion in US is 0.6
    initI = one_county.iloc[0,1]*0.4
    initR = one_county.iloc[0,1]*0.6
    initial_conditions = [initE, initI, initR, initN]
    
    # Init parameters
    beta = 0.3
    sigma = 0.8
    gamma = 0.6
    params['beta'].value = beta
    params['sigma'].value = sigma
    params['gamma'].value = gamma
    days = 30
    tspan = np.arange(0, days, 1)
    data = one_county.loc[0:(days-1), 'daily_confirmed'].values.astype(np.float)
    
    # use squred error
    def error(params, initial_conditions, tspan, data):
        sol = ode_solver(tspan, initial_conditions, params)

        return (sol[:,2]+sol[:,3]-data).ravel()
    
    result = minimize(error, params, args=(initial_conditions, tspan, data), method='leastsq')
    observed_IR = one_county.loc[:, 'daily_confirmed'].values
    
    tspan_fit_pred = np.arange(0, observed_IR.shape[0], 1)
    params['beta'].value = result.params['beta'].value
    params['sigma'].value = result.params['sigma'].value
    params['gamma'].value = result.params['gamma'].value
    
    # estimate params and return the fit values 
    fitted_predicted = ode_solver(tspan_fit_pred, initial_conditions, params)
    # Get I and R cols
    fitted_predicted_IR = fitted_predicted[:, 2:4]
    
    # calculate fit MAPE error
    a = np.abs(fitted_predicted_IR[:days, 0]+fitted_predicted_IR[:days, 1] - observed_IR[:days])
    b = observed_IR[:days]
    c = div0(a,b)
    ferror = np.mean(c)
    fit_error.append(ferror)
    
    # calculate pred MAPE error
    pred_IR = fitted_predicted_IR[days:observed_IR.shape[0], 0]+fitted_predicted_IR[days:observed_IR.shape[0], 1]
    a = np.abs(pred_IR - observed_IR[days:])
    b = observed_IR[days:]
    c = div0(a,b)
    perror = np.mean(c)
    
    # Get naive error
    naive_error = naive_model_per_county[naive_model_per_county['fip']==fip]['iter_pred_error'].values[0]
    

    #  Collect this county's daily preds     
    one_county_preds = one_county.iloc[days:,:].reset_index(drop=True)
    one_county_preds['County Name'] = name
    one_county_preds['State'] = state
    one_county_preds['fip'] = fip
    one_county_preds['seir_orig_pred'] = pred_IR
    
    # if the fit error <1
    if ferror<1:
        # use SEIR model's prediction
        ens_error = perror
        one_county_preds['seir_ensemble_pred'] = pred_IR
    else:
        # use naive model's prediction
        ens_error = naive_error
        one_county_preds['seir_ensemble_pred'] = naive_daily[naive_daily['fip']==fip]['iter_pred'].values
    
    #  append the resuts
    per_fip.append(fip)
    per_county.append(name)
    per_state.append(state)
    pred_error.append(perror)
    ensemble_error.append(ens_error)
    all_preds_test = pd.concat([all_preds_test,one_county_preds])

In [ ]:
# Create Data Frame
seir_per_county = pd.DataFrame(columns=['fip', 'County Name', 'State','seir_pred_error','fit_error','ensemble_error'])
seir_per_county['seir_pred_error'] = pred_error
seir_per_county['fit_error'] = fit_error
seir_per_county['fip'] = per_fip
seir_per_county['County Name'] = per_county
seir_per_county['State'] = per_state
seir_per_county['ensemble_error'] =ensemble_error
seir_per_county.head()

,fip,County Name,State,seir_pred_error,fit_error,ensemble_error
0,1001,Autauga County,AL,0.052650,0.004449,0.052650
1,1003,Baldwin County,AL,0.064356,0.010308,0.064356
2,1005,Barbour County,AL,0.044818,0.004666,0.044818
3,1007,Bibb County,AL,0.055055,0.004222,0.055055
4,1009,Blount County,AL,0.025497,0.006341,0.025497


In [ ]:
all_preds_test.head()

,fip,County Name,State,date,seir_orig_pred,seir_ensemble_pred,daily_confirmed
0,1001,Autauga County,AL,10/1/20,1828.483537,1828.483537,1798
1,1001,Autauga County,AL,10/2/20,1843.327700,1843.327700,1805
2,1001,Autauga County,AL,10/3/20,1858.286289,1858.286289,1818
3,1001,Autauga County,AL,10/4/20,1873.359617,1873.359617,1828
4,1001,Autauga County,AL,10/5/20,1888.547997,1888.547997,1831


In [ ]:
seir_per_county.to_csv(path +'data/seir_per_county.csv')
all_preds_test.to_csv(path +'data/seir_all_preds_test.csv')